## 1.1 날짜 추출

- td 태그중에 원하는 정보만을 따로 가져와야 합니다. (표를 자세히 보시면 왜 54개인지 알 수 있습니다.)


- 날짜 태그의 규칙을 찾아서 td 태그들 중에 날짜를 가져옵니다.


- 그런데 해당 페이지위에서 날짜 정보가 있는 tag에 대한 규칙을 찾기가 어렵습니다. 이럴 때 XPath 정보를 이용하여 바로 검색할 태그를 지정할 수 있습니다.


- 아래 그림과 같이 우클릭을 해서 Copy XPath를 눌러줍니다.

![sise_xpath](webcrawling/get_xpath.png)

In [ ]:
import requests
import bs4

page_no = 1
page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}'
page_url

source = requests.get(page_url).text
print(source)

source = bs4.BeautifulSoup(source)
print(source)

In [17]:
# date에 대한 xpath
#/html/body/div/table[1]/tbody/tr[3]/td[1] 

dates = source.find_all('td',class_ = 'date')
dates

[<td class="date">2024.02.15</td>,
 <td class="date">2024.02.14</td>,
 <td class="date">2024.02.13</td>,
 <td class="date">2024.02.08</td>,
 <td class="date">2024.02.07</td>,
 <td class="date">2024.02.06</td>]

In [18]:
dates[0].text

'2024.02.15'

In [24]:
# tag를 통해 가져온 다음에 텍스트를 가져옵니다. 데이터는 text 형태로 tag 사이에 있습니다.
# tag 사이에 껴있는 텍스트를 가져올 때 .text를 사용합니다.

date_list =[]

for date in dates:
    date_list.append(date.text)

date_list


['2024.02.15',
 '2024.02.14',
 '2024.02.13',
 '2024.02.08',
 '2024.02.07',
 '2024.02.06']

In [ ]:
# XPath

/html/body/div/table[1]/tbody/tr[3]/td[1]

- 날짜를 크롤링하는데 성공했습니다!!

## 1.2 체결가(종가) 추출

#### 페이지 상의 날짜와 종가정보 전체 추출

In [31]:
prices = source.find_all('td',class_ = 'number_1') 
prices[::4]         # 처음부터 끝까지 중 4칸씩

[<td class="number_1">351.21</td>,
 <td class="number_1">352.69</td>,
 <td class="number_1">357.38</td>,
 <td class="number_1">353.29</td>,
 <td class="number_1">352.25</td>,
 <td class="number_1">347.80</td>]

In [32]:
price_list = []
for price in prices[::4]:
    price_list.append(price.text)

price_list

['351.21', '352.69', '357.38', '353.29', '352.25', '347.80']

In [34]:
len(date_list)

6

In [35]:
len(price_list)

6

- 페이지에서 표의 태그들을 살펴보면, 체결가 이후로 전일비/등락률/거래량/거래대금 들도 같은 태그를 공유하고 있어, 4개씩 증가하는 것을 알 수 있습니다.


- 이것을 규칙으로 number_1으로 추출한 태그는 24개이지만, 이 중에서 4의 배수로 건너뛰면서 추출하면 바로 체결가를 가져올 수 있다는 것을 확인할 수 있습니다.

- 이제 크롤러의 마지막 관문이 남았습니다


- 현재까지 하나의 페이지에서 크롤링 하는 것을 완료하였습니다.


- 이제 이 작업을 원하는 페이지까지(또는 끝까지) **반복** 진행하면 됩니다.


- 그렇다면 이제 마지막 페이지를 찾을 차례입니다.

#### 마지막 페이지 번호 찾기

![last_page](webcrawling/get_last_page.png)

In [40]:
# 역시 해당 tag의 xpath를 복사해옵니다. 하지만 규칙이 간단해서 xpath를 굳이 사용하진 않겠습니다.

# /html/body/div/table[2]/tbody/tr/td[7]/a

# td tag중에 class가 pgRR인 태그를 찾아서, 그 하위에 있는 a tag의 href 속성값을 가져옵니다.

last_url = source.find_all('td',class_ = 'pgRR')[0].find_all('a')[0]['href']
last_url

'/sise/sise_index_day.naver?code=KPI200&page=747'

- 위에 최종적으로 가져온 문자열에서 마지막 3글자를 때어내면 우리가 원하는 마지막 페이지 숫자인 591이 나옵니다.

In [41]:
# 마지막 글자를 때어내기 위해서 규칙을 생각해봅니다.
last_url.split('&page=')

['/sise/sise_index_day.naver?code=KPI200', '747']

In [45]:
last_page = last_url.split('&page=')[-1]
last_page

'747'

- 진짜 마지막 단계입니다. 해당 페이지 주소의 규칙을 찾아야 합니다.


- 이제 크롤링의 과정은 다음과 같습니다.

1) 일별 시세 정보가 있는 페이지에 접속한다.


2) 페이지에서 날짜 / 체결가가 들어있는 태그를 검색한다.


3) 태그 중에서 우리가 찾은 조건에 맞는 데이터만 가져온다.


4) 이를 원하는만큼(현재는 마지막페이지까지) 반복한다.

In [ ]:
# 100 페이지에 접속하는 예시
naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(100)

# 위에서 작성했던 모든 코드를 종합합니다.



## 1.3 가져온 모든 정보를 하나로 합쳐서 구현하기 

In [49]:
# 위에서 구현한 모든 내용을 하나의 코드로 구현합니다.
date_list = []
price_list = []

for page_no in range(1,int(last_page)+1):
    page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}'

    source = requests.get(page_url).text
    source = bs4.BeautifulSoup(source)

    dates = source.find_all('td',class_ ='date')

    for date in dates:
        date_list.append(date.text)
    
    prices = source.find_all('td',class_ ='number_1')


    for price in prices[::4]:
        price_list.append(price.text)


In [51]:
len(date_list),len(price_list)

(4482, 4482)

In [56]:
import pandas as pd

df = pd.DataFrame({'date': date_list,
                  'price': price_list}).dropna()
df

,date,price
0,2024.02.15,351.21
1,2024.02.14,352.69
2,2024.02.13,357.38
3,2024.02.08,353.29
4,2024.02.07,352.25
...,...,...
4477,,
4478,,
4479,,
4480,,


In [57]:
df.to_excel('kpi200.xlsx',index=False)